In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def calculate_F(X_star, X, y, alpha, beta):
    result = []
    
    n_samples = len(X)

    for i in range(n_samples):
        # calculate EI (equation 2)
        E = alpha[i] * y[i] * np.dot(X[i], X_star)
        result.append(E)

        fx = np.sum(result) + beta
        
        return fx

In [3]:
def calculate_E(X_star,y_star, X, y, alpha, beta):
    # Ei = f(xi) - yi
    fx = calculate_F(X_star, X, y, alpha, beta)
    
    # calculate Ei
    Ei = fx - y_star
    
    return Ei

In [4]:
def generate_random_j(n_samples, i):
    j = np.random.choice(n_samples)
    
    if j != i:
        return j
    else:
        generate_random_j(n_samples, i)

In [5]:
def cal_boundary(y_i, y_j, a_i, a_j, C):
    
    if y_i != y_j:
        L = max(0, a_j - a_i)
        H = min(C, C + a_j - a_i)
    else:
        L = max(0, a_i + a_j - C)
        H = min(C, a_i + a_j)
        
    return L, H

In [6]:
# equation 14
def cal_eta(X_i, X_j):
    
    eta = (2 * np.dot(X_i, X_j)) - np.dot(X_i, X_i) - np.dot(X_j, X_j)
    
    return eta

In [7]:
dictt = {
    "X1":[0,0,1,1],
    "X2":[0,1,0,1],
    "y":[-1,1,1,1]
}

df = pd.DataFrame(dictt)
df

,X1,X2,y
0,0,0,-1
1,0,1,1
2,1,0,1
3,1,1,1


In [8]:
C = 0.1
tol = 1e-6
max_passes = 10

In [9]:
X = np.array(df[["X1", "X2"]])
y = np.array(df["y"])

In [10]:
X

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [11]:
y

array([-1,  1,  1,  1])

In [12]:
n_samples = len(X)

# initialize
alpha = np.zeros(n_samples)
beta = 0
passes = 0

In [13]:
n_samples

4

In [33]:
while passes > max_passes:
    mum_changed_alphas = 0
    
    for i in range(n_samples):
        # extract data index i
        X_i = X[i]
        y_i = y[i]
        a_i = alpha[i]
        
        # hitung ei
        Ei = calculate_E(X_star = X_i, 
                         y_star = y_i,
                         X = X, 
                         y = y, 
                         alpha = alpha, 
                         beta = beta)
        
        cond_1 = ((y_i*Ei) < -tol) and (a_i < C)
        cond_2 = ((y_i*Ei) > tol) and (a_i > 0)
        
        if cond_1 or cond_2:
            # select random j != i
            j = generate_random_j(n_samples=n_samples, 
                                  i =i ) 
        
            # extract data index i
            X_j = X[j]
            y_j = y[j]
            a_j = alpha[j]

            # hitung E_j
            Ej = calculate_E(X_star = X_j,
                             y_star = y_j,
                             X = X,
                             y = y,
                             alpha = alpha,
                             beta = beta)


            # update alpha old
            alpha_i_old = a_i
            alpha_j_old = a_j

            # compute L and H (equation 10 and 11)
            L, H = cal_boundary(y_i = y_i, 
                                y_j = y_j, 
                                a_i = a_i, 
                                a_j = a_j, 
                                C = C)

            if L == H:
                continue

            # equation 14
            eta = cal_eta(X_i = X_i, 
                          X_j = X_j)

            if eta >= 0:
                continue

            # compute and clip aj using equation 12 and 15

            a_j -= (y_j * (E_i - E_j))/eta

            # condition clip new value of j
            if a_j > H:
                a_j = H
                
            elif L <= a_j and a_j <= H:
                a_j = a_j
                
            else:
                a_j = L

            if np.abs(a_j - alpha_j_old) < tol:
                continue

            # equation 16
            a_i += (y_i * y_j)*(alpha_j_old - a_j)

            # calculate b1 and b2 (equation 17 and 18)
            b1 = beta - Ei \
                 - y_i * (a_i - alpha_i_old) * np.dot(X_i, X_i) \
                 - y_j * (a_j - alpha_j_old) * np.dot(X_i, X_j)

            b2 = beta - Ej \
                 - y_i * (a_i - alpha_i_old) * np.dot(X_i, X_j) \
                 - y_j * (a_j - alpha_j_old) * np.dot(x_j, x_j)

            # compute b with equation 19
            if 0 < a_i and a_i < C:
                beta = b1

            elif 0 < a_j and a_j < C:
                beta = b2

            else:
                beta = (b1+b2)/2

            alpha[i] = a_i
            alpha[j] = a_j
            
            num_changed_alphas =+ 1
            
        
    if num_changed_alphas == 0:
        passes += 1
        
    else:
        passes += 0

In [36]:
alpha

array([0., 0., 0., 0.])

In [37]:
beta

0

In [17]:
def calculate_w(alpha, X, y):
    w = np.dot(X.T, alpha*y)
    return w

In [23]:
w = calculate_w(alpha = alpha, X=X, y=y)
w

array([0., 0.])

In [31]:
def predict(X, w, b):
    y_pred = np.dot(w, X) + b
    
    return np.sign((y_pred>0)).astype(int)